In [27]:
#import the following packages to work with json, pandas, glob,os
import json
import pandas as pd
from pandas.io.json import json_normalize
import os
from glob import glob
from os.path import join, abspath
from os import listdir, getcwd

In [28]:
#provide the absolute path of the json files.
dir_path = "C:\Users\Raghuveera\Notebooks\Notebooks\jsons"
#full_path will have the path of the files which has .json
full_path = join(abspath(getcwd()), dir_path, "*.json")
#create an empty dataframe to append each hotelInfo after every loop
resultFrame = pd.DataFrame()

In [29]:
#loop through each json file and extract hotelInfo, Reviews
for fili in glob(full_path):
#open the file and load the file in to a dataframe mydata
        with open(fili) as f:
            mydata = pd.DataFrame(json.loads(line) for line in f)
#loop mydata dataframe and get HotelInfo and append to df_hotelInfo
            df_hotelInfo = pd.concat([pd.DataFrame.
                                      from_dict(item, orient='index').
                                      T for item in mydata.HotelInfo])
#loop mydata dataframe and get Reviews and append the reviews to df dataframe
            df = pd.concat([pd.DataFrame.
                            from_records(item).
                            T for item in mydata.Reviews])
#create df_reviews dataframe by transposing
            df_reviews = df.transpose()
#check for name and hotelID. If no name,give an empty string
            if 'Name' in df_hotelInfo.columns:
                df_reviews['Name']=df_hotelInfo['Name']
            else:
                df_reviews['Name']= " "
                
            if 'HotelID' in df_hotelInfo.columns:
                df_reviews['HotelID'] = df_hotelInfo['HotelID']
            else:
                df_reviews['HotelID'] = " "
#convert the ratings in to columns for each review in df_reviews
            df_reviewsAsCols = pd.concat([df_reviews.
                                          drop(['Ratings'], axis=1), 
                                          df_reviews['Ratings'].
                                          apply(pd.Series)], axis=1)
# get the required columns from df_reviewsAsCols dataframe as per part 1
            df_hotel1 = df_reviewsAsCols.loc[
                :,['Name','HotelID','Author','Date','ReviewID',
                   'Cleanliness','Location','Overall','Rooms',
                   'Service','Sleep Quality','Value',
                   'Business service (e.g., internet access)',
                   'Check in / front desk','Content']]
#for each review assign the hotelname and ID.
            df_hotel1['Name'] =  df_hotel1[
                'Name'].apply(lambda x:df_reviews['Name'])
            df_hotel1['HotelID'] =  df_hotel1[
                'HotelID'].apply(lambda x:df_reviews['HotelID'])  
#append the results to initially created empty dataframe resultFrame
            resultFrame = resultFrame.append(df_hotel1, ignore_index=True)
#fill with 0's where ever NaN is present.            
resultFrame = resultFrame.fillna(0)
#print the count of rows and columns
print "No of Rows -  ,No of Column -  "
countRows = resultFrame.shape[0]
countColumns = resultFrame.shape[1]
print countRows
print countColumns

No of Rows -  ,No of Column -  
2485
15


In [30]:
resultFrame

,Name,HotelID,Author,Date,ReviewID,Cleanliness,Location,Overall,Rooms,Service,Sleep Quality,Value,"Business service (e.g., internet access)",Check in / front desk,Content
0,Hotel Seattle,100506,luvsroadtrips,"January 3, 2012",UR122476164,1,5,1.0,1,1,1,1,0,0,This place is not even suitable for the homele...
1,Hotel Seattle,100506,estelle e,"December 29, 2011",UR122239883,4,5,4.0,3,4,5,3,0,0,We stayed in downtown hotel Seattle for two ni...
2,Hotel Seattle,100506,RobertEddy,"December 20, 2011",UR121931325,2,1,1.0,1,1,1,1,0,0,"i made reservations and when i showed up, i qu..."
3,Hotel Seattle,100506,James R,"October 30, 2011",UR119896310,1,1,1.0,1,1,1,1,0,0,This hotel is so bad it's a joke. I could bare...
4,Hotel Seattle,100506,Shobha49,"September 14, 2011",UR118110693,0,5,1.0,1,1,0,3,0,0,My husband and I stayed at this hotel from 16t...
5,Hotel Seattle,100506,PalomitaBlanca,"September 6, 2011",UR117757669,4,0,3.0,0,4,3,3,0,0,Very basic hotel and good rate compared to mos...
6,Hotel Seattle,100506,Qualnurse,"September 6, 2011",UR117730450,4,0,4.0,0,4,4,5,0,0,"We stayed for one night, before a cruise to Al..."
7,Hotel Seattle,100506,Debbie R,"August 27, 2011",UR117303221,2,0,1.0,0,2,2,1,0,0,Stay here if you want to experenice hell. Joel...
8,Hotel Seattle,100506,travelitch_gal,"August 23, 2011",UR117119391,4,0,3.0,0,4,3,4,0,0,If you are looking for a value for money hotel...
9,Hotel Seattle,100506,Khaap,"August 10, 2011",UR116501344,5,0,4.0,0,4,5,5,0,0,This hotel had character! I really liked our r...


In [31]:
#Part1)b-add a counter column and do a 
#group by to get the no of reviews per hotel
resultFrame['COUNTER'] = 1  
df = resultFrame.groupby(['HotelID'])['COUNTER'].sum() 
df

HotelID
100506      48
1217974      1
150849     710
214680     174
240124      52
2515575     96
287670     188
550994     167
655424       1
677703       1
72572      233
72579      113
72586      137
72598       36
73393       93
73644       54
73706       45
73712       25
73718       57
73727       44
73739       60
73743       32
73751       19
73757       36
73760       39
73768       24
Name: COUNTER, dtype: int64

In [32]:
import numpy as np
#Part1.c) calculate the requested summary statistics
#make sure all the ratings in Overall column are numeric
resultFrame['Overall']=resultFrame['Overall'].apply(pd.to_numeric)
resultFrame['Overall'].describe()

count    2485.000000
mean        3.728370
std         1.300262
min         1.000000
25%         3.000000
50%         4.000000
75%         5.000000
max         5.000000
Name: Overall, dtype: float64

In [33]:
#pickling -the dataframe
resultFrame.to_pickle('resultFrameP')

In [34]:
#Part 2
#get the content from the resultFrame.content has all the reviews/hotel
resultFrame['Content']
#combine all the reviews for each hotel to form a single string per Hotel
review_content = resultFrame.groupby('HotelID')['Content'].apply(
    ' '.join).reset_index()
#review_content.get

In [35]:
# Import BeautifulSoup into your workspace
from bs4 import BeautifulSoup
#import nltk to download the stop words
import nltk
#donwload the stopwords from nltk, Download text data sets, including stop words
nltk.download() 
#from nltk, get the list of stopwords
from nltk.corpus import stopwords # Import the stop word list
from collections import Counter
import re

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [36]:
#get the list of stopwords from nltk tool kit for english
stopwords = stopwords.words("english")
#Create a text file of stopwords which include more stopwords
#append your own list of stopwords to nltk's stopwords
with open('stopwords.txt', 'r') as myfile:
    allStopwords = [line.rstrip('\n') for line in open('stopwords.txt')]    
    allStopwords.append(stopwords)

In [37]:
#create a function to be called on each column of content
def parseReviews(x):
    hotelReviews = BeautifulSoup(x, 'lxml') 
    nomral_text = re.sub(
        r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*',
        '', hotelReviews.get_text())
# The pattern to search for, The pattern to replace it with
    letters_only = re.sub("[^a-zA-Z]",           
                      " ",                   
                      nomral_text)
# Convert to lower case
    lower_case = letters_only.lower()   
#split the words
    words = lower_case.split()
#remove stopwords
    words = [w for w in words if not w in allStopwords] 
#get the counter on each word
    worddict = Counter(words)
    return worddict

In [38]:
#create an empty dict hotwords
hotwords = {}
#call the function parseReviews on Content
#aplly the returned worddict to each column of hotelID
review_content['words'] = review_content['Content'].apply(
    lambda x: parseReviews(x))
#add the hotel_id, hotwords to a dataframe 
hotwords_df = review_content.loc[:,['HotelID','words']]

In [39]:
#make the hotelID index column before converting to json file.
hotwords_df.set_index("HotelID", drop=True, inplace=True)

In [40]:
#this is a dictionary of dictionarys which has hotelID and content count.
dictionary = hotwords_df.to_dict(orient="index")

In [41]:
# Getting keys of the dict
dictionary.keys()

[u'214680',
 u'73393',
 u'73751',
 u'72586',
 u'73757',
 u'72572',
 u'73644',
 u'150849',
 u'73768',
 u'655424',
 u'287670',
 u'240124',
 u'73727',
 u'73743',
 u'1217974',
 u'73706',
 u'677703',
 u'72598',
 u'72579',
 u'73760',
 u'2515575',
 u'100506',
 u'550994',
 u'73718',
 u'73739',
 u'73712']

In [42]:
#getting values of the dict 
dictionary.values()

[{'words': Counter({u'limited': 2,
           u'heplfull': 1,
           u'personally': 2,
           u'yellow': 2,
           u'catch': 5,
           u'sleep': 12,
           u'railing': 1,
           u'hotelwas': 1,
           u'saved': 1,
           u'trolley': 2,
           u'buddy': 1,
           u'visable': 1,
           u'remodeled': 2,
           u'patched': 1,
           u'teaching': 1,
           u'worth': 10,
           u'updated': 3,
           u'blanket': 1,
           u'utmost': 1,
           u'bringing': 1,
           u'tickets': 3,
           u'school': 4,
           u'basics': 1,
           u'leonor': 16,
           u'whichmakes': 1,
           u'companies': 2,
           u'humidity': 1,
           u'convenience': 5,
           u'deadline': 1,
           u'heading': 3,
           u'triumph': 1,
           u'clothes': 1,
           u'enjoy': 10,
           u'tired': 6,
           u'corvette': 1,
           u'hanging': 2,
           u'direct': 2,
           u'surrounding

In [43]:
#dump the dict in to json file
j = json.dumps(dictionary, indent=4)
f = open('hotwords.json', 'w')
print >> f, j
f.close()